Now we'll look at a more interesting and clearly nonlinear example -- inferring an unknown conductivity rather than an unknown right-hand side.

In [ ]:
import firedrake
import firedrake_adjoint
mesh = firedrake.UnitSquareMesh(32, 32)
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

In [ ]:
from firedrake import Constant, cos, sin
import numpy as np
from numpy import pi as π
from numpy import random

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])

        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z
        
        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)
        
        q_true += mode

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

Compute the true solution of the PDE.

In [ ]:
from firedrake import exp, inner, grad, dx
u = firedrake.Function(V)
f = Constant(1.0)
J = (0.5 * exp(q_true) * inner(grad(u), grad(u)) - f * u) * dx
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = firedrake.derivative(J, u)
firedrake.solve(F == 0, u, bc)
u_true = u.copy(deepcopy=True)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

Generate the observational data.

In [ ]:
num_points = 50
δs = np.linspace(-0.5, 2, num_points + 1)
X, Y = np.meshgrid(δs, δs)
xs = np.vstack((X.flatten(), Y.flatten())).T

θ = π / 12
R = np.array([
    [np.cos(θ), -np.sin(θ)],
    [np.sin(θ), np.cos(θ)]
])

xs = np.array([
    x for x in (xs - np.array([0.5, 0.5])) @ R
    if (0 <= x[0] <= 1) and (0 <= x[1] <= 1)
])

Synthesize some observational data.

In [ ]:
U = u_true.dat.data_ro[:]
u_range = U.max() - U.min()
signal_to_noise = 20
σ = firedrake.Constant(u_range / signal_to_noise)
ζ = generator.standard_normal(len(xs))
u_obs = np.array(u_true.at(xs)) + float(σ) * ζ

point_cloud = firedrake.VertexOnlyMesh(mesh, xs)
Z = firedrake.FunctionSpace(point_cloud, 'DG', 0)
u_o = firedrake.Function(Z)
u_o.dat.data[:] = u_obs

Start with an initial guess of $q = 0$.

In [ ]:
u = firedrake.Function(V)
q = firedrake.Function(Q)
J = (0.5 * exp(q) * inner(grad(u), grad(u)) - f * u) * dx
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = firedrake.derivative(J, u)
firedrake.solve(F == 0, u, bc)

Very different!

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u, axes=axes, shading='gouraud')
fig.colorbar(colors);

Form the objective functional.

In [ ]:
Πu = firedrake.interpolate(u, Z)
E = 0.5 * ((u_o - Πu) / σ)**2 * dx

In [ ]:
α = firedrake.Constant(0.5)
R = 0.5 * α**2 * inner(grad(q), grad(q)) * dx

In [ ]:
J = firedrake.assemble(E) + firedrake.assemble(R)

Create the reduced functional and minimize the objective.

In [ ]:
q̂ = firedrake_adjoint.Control(q)
Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)

In [ ]:
q_min = firedrake_adjoint.minimize(Ĵ, method='Newton-CG', options={'disp': True})

In [ ]:
fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True)
for ax in axes:
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)

kw = {'vmin': -5, 'vmax': +5, 'shading': 'gouraud'}
axes[0].set_title('Estimated')
firedrake.tripcolor(q_min, axes=axes[0], **kw)
axes[1].set_title('True')
firedrake.tripcolor(q_true, axes=axes[1], **kw);